In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
from pynq import Overlay
overlay = Overlay('./bitstream/DEC_rLUT20bits.bit')

In [3]:
overlay.ip_dict

{'DEC_rLUT20bits_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xaf821690>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'DEC_rLUT20bits_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S00_AXI',
  'parameters': {'C_S00_AXI_ADDR_WIDTH': '4',
   'C_S00_AXI_BASEADDR': '0x43C00000',
   'C_S00_AXI_DATA_WIDTH': '32',
   'C_S00_AXI_HIGHADDR': '0x43C0FFFF',
   'Component_Name': 'DEC_rLUT20bits_DEC_rLUT20bits_0_0',
   'EDK_IPTYPE': 'PERIPHERAL'},
  'phys_addr': 1136656384,
  'registers': {},
  'state': None,
  'type': 'xilinx.com:user:DEC_rLUT20bits:1.0'}}

In [4]:
from pynq import MMIO
ip = MMIO(0x43C00000, 0x10000)

In [5]:
ip.device

### 將 r_LUT  的location輸出值轉為2補數:

In [6]:
def twos_complement_32bit(x):
    """將 32-bit 補數整數轉為有號值"""
    if x >= (1 << 31):
        return x - (1 << 32)
    else:
        return x

#### Test:

In [7]:
remainder_1 = 1868

ip.write_mm(0, remainder_1)

location_1 = ip.read_mm(4)
location_2 = ip.read_mm(8)


print(twos_complement_32bit(location_1))
print(twos_complement_32bit(location_2))

-6
22


### 開始則中查表驗證

## 折衷查表:
### 用 Single Error Correction 解 Double Error Correction (A 取Double_AWE的 A)

#### 1.取 A的bits $\approx$ N的bits $\Rightarrow$ A = 18613  (30bits), N的範圍: 0 ~ 1073741823 (30bits)

### Define function
* (Not including 2's complement range

In [8]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return unique_list


def get_sign(number):
    return 1 if number > 0 else -1 if number < 0 else 0


print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


In [9]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data1 = (1 << 20) - 11    ## 
data2 = (1 << 20) - 19
data3 = (1 << 20) - 15
AN = 6311
input1 = data1*AN
input2 = data2*AN
input3 = data3*AN
bits1 = bits_needed(input1)
bits2 = bits_needed(input2)
bits3 = bits_needed(input3)
print("inputs1的位元數:", bits1)
print("inputs2的位元數:", bits2)
print("inputs3的位元數:", bits3)

inputs1的位元數: 33
inputs2的位元數: 33
inputs3的位元數: 33


### 產生1-bit AWE (Arithmetic-Weight Error)
* 並產生 Delta Q 的 list

In [10]:
AWE1_1=[]
e1_1=0     ## 錯誤數量
e1_data1=[]
h = [];   #位置
for i in range(0, bits1):
    e1_data1.insert(e1_1 , input1 + 2**i)
    AWE1_1.insert(e1_1, 2**i)
    e1_1 = e1_1 + 1
    e1_data1.insert(e1_1 , input1 - 2**i)
    AWE1_1.insert(e1_1, -2**i)
    e1_1 = e1_1 + 1
    h.insert(i,i+1)

AWE1_2=[]
e1_2=0     ## 錯誤數量
e1_data2=[]
for i in range(0, bits2):
    e1_data2.insert(e1_2 , input2 + 2**i)
    AWE1_2.insert(e1_2, 2**i)
    e1_2 = e1_2 + 1
    e1_data2.insert(e1_2 , input2 - 2**i)
    AWE1_2.insert(e1_2, -2**i)
    e1_2 = e1_2 + 1
    h.insert(i,i+1)
    
AWE1_3=[]
e1_3=0     ## 錯誤數量
e1_data3=[]
for i in range(0, bits3):
    e1_data3.insert(e1_3 , input3 + 2**i)
    AWE1_3.insert(e1_3, 2**i)
    e1_3 = e1_3 + 1
    e1_data3.insert(e1_3 , input3 - 2**i)
    AWE1_3.insert(e1_3, -2**i)
    e1_3 = e1_3 + 1
    h.insert(i,i+1)

h_awe = [val for num in h for val in (num, -num)]
print("產生1-bit AWE:", AWE1_1)
print("不同輸入資料的AWE是否相等:",AWE1_1 == AWE1_2 == AWE1_3)
print(h_awe)
print(np.size(AWE1_1))

產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296]
不同輸入資料的AWE是否相等: True
[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -20, 21, -21, 22, -22, 23, -23, 24, -24, 25, -25, 26, -26, 27, -27, 28, -28, 29, -29, 30, -30, 31, -31, 32, -32, 33, -33, 1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -20, 21, 

### 產生SEC的模數:

In [11]:
SEC_R = []
AWE1_R= []
for m in range(0,e1_1):
    mod = e1_data1[m] % AN
    SEC_R.append(mod)

for n in range(0,e1_1):
    mod_awe = AWE1_3[n] % AN
    AWE1_R.append(mod_awe)


print("SEC的模數:\n",SEC_R)
print("AWE1的模數:\n",SEC_R)
print("AWE1的模數是否等於SEC模數:", SEC_R == AWE1_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 6310, 2, 6309, 4, 6307, 8, 6303, 16, 6295, 32, 6279, 64, 6247, 128, 6183, 256, 6055, 512, 5799, 1024, 5287, 2048, 4263, 4096, 2215, 1881, 4430, 3762, 2549, 1213, 5098, 2426, 3885, 4852, 1459, 3393, 2918, 475, 5836, 950, 5361, 1900, 4411, 3800, 2511, 1289, 5022, 2578, 3733, 5156, 1155, 4001, 2310, 1691, 4620, 3382, 2929, 453, 5858, 906, 5405, 1812, 4499, 3624, 2687]
AWE1的模數:
 [1, 6310, 2, 6309, 4, 6307, 8, 6303, 16, 6295, 32, 6279, 64, 6247, 128, 6183, 256, 6055, 512, 5799, 1024, 5287, 2048, 4263, 4096, 2215, 1881, 4430, 3762, 2549, 1213, 5098, 2426, 3885, 4852, 1459, 3393, 2918, 475, 5836, 950, 5361, 1900, 4411, 3800, 2511, 1289, 5022, 2578, 3733, 5156, 1155, 4001, 2310, 1691, 4620, 3382, 2929, 453, 5858, 906, 5405, 1812, 4499, 3624, 2687]
AWE1的模數是否等於SEC模數: True
SEC的大小: 66


### 產生2-bit AWE (Arithmetic-Weight Error)

In [12]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
for i in range(0, bits1):
    for j in range(i+1, bits1):
        e2_data.insert(e2 , (input1 + 2**i + 2**j))
        AWE_2.insert(e2 , (2**i + 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 + 2**i - 2**j))
        AWE_2.insert(e2 , (2**i - 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i + 2**j))
        AWE_2.insert(e2 , (-2**i + 2**j))                
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i -2**j))
        AWE_2.insert(e2 , (-2**i - 2**j))
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        p = p + 1


print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits 錯誤資料:",e2_data)

第 0 筆錯誤資料: 6617493718
第 0 筆算術錯誤: 3
第 1 筆錯誤資料: 6617493714
第 1 筆算術錯誤: -1
第 2 筆錯誤資料: 6617493716
第 2 筆算術錯誤: 1
第 3 筆錯誤資料: 6617493712
第 3 筆算術錯誤: -3
第 4 筆錯誤資料: 6617493720
第 4 筆算術錯誤: 5
第 5 筆錯誤資料: 6617493712
第 5 筆算術錯誤: -3
第 6 筆錯誤資料: 6617493718
第 6 筆算術錯誤: 3
第 7 筆錯誤資料: 6617493710
第 7 筆算術錯誤: -5
第 8 筆錯誤資料: 6617493724
第 8 筆算術錯誤: 9
第 9 筆錯誤資料: 6617493708
第 9 筆算術錯誤: -7
第 10 筆錯誤資料: 6617493722
第 10 筆算術錯誤: 7
第 11 筆錯誤資料: 6617493706
第 11 筆算術錯誤: -9
第 12 筆錯誤資料: 6617493732
第 12 筆算術錯誤: 17
第 13 筆錯誤資料: 6617493700
第 13 筆算術錯誤: -15
第 14 筆錯誤資料: 6617493730
第 14 筆算術錯誤: 15
第 15 筆錯誤資料: 6617493698
第 15 筆算術錯誤: -17
第 16 筆錯誤資料: 6617493748
第 16 筆算術錯誤: 33
第 17 筆錯誤資料: 6617493684
第 17 筆算術錯誤: -31
第 18 筆錯誤資料: 6617493746
第 18 筆算術錯誤: 31
第 19 筆錯誤資料: 6617493682
第 19 筆算術錯誤: -33
第 20 筆錯誤資料: 6617493780
第 20 筆算術錯誤: 65
第 21 筆錯誤資料: 6617493652
第 21 筆算術錯誤: -63
第 22 筆錯誤資料: 6617493778
第 22 筆算術錯誤: 63
第 23 筆錯誤資料: 6617493650
第 23 筆算術錯誤: -65
第 24 筆錯誤資料: 6617493844
第 24 筆算術錯誤: 129
第 25 筆錯誤資料: 6617493588
第 25 筆算術錯誤: -127
第 26 筆錯誤資料: 6617493842
第 26 筆

第 380 筆錯誤資料: 6617493787
第 380 筆算術錯誤: 72
第 381 筆錯誤資料: 6617493659
第 381 筆算術錯誤: -56
第 382 筆錯誤資料: 6617493771
第 382 筆算術錯誤: 56
第 383 筆錯誤資料: 6617493643
第 383 筆算術錯誤: -72
第 384 筆錯誤資料: 6617493851
第 384 筆算術錯誤: 136
第 385 筆錯誤資料: 6617493595
第 385 筆算術錯誤: -120
第 386 筆錯誤資料: 6617493835
第 386 筆算術錯誤: 120
第 387 筆錯誤資料: 6617493579
第 387 筆算術錯誤: -136
第 388 筆錯誤資料: 6617493979
第 388 筆算術錯誤: 264
第 389 筆錯誤資料: 6617493467
第 389 筆算術錯誤: -248
第 390 筆錯誤資料: 6617493963
第 390 筆算術錯誤: 248
第 391 筆錯誤資料: 6617493451
第 391 筆算術錯誤: -264
第 392 筆錯誤資料: 6617494235
第 392 筆算術錯誤: 520
第 393 筆錯誤資料: 6617493211
第 393 筆算術錯誤: -504
第 394 筆錯誤資料: 6617494219
第 394 筆算術錯誤: 504
第 395 筆錯誤資料: 6617493195
第 395 筆算術錯誤: -520
第 396 筆錯誤資料: 6617494747
第 396 筆算術錯誤: 1032
第 397 筆錯誤資料: 6617492699
第 397 筆算術錯誤: -1016
第 398 筆錯誤資料: 6617494731
第 398 筆算術錯誤: 1016
第 399 筆錯誤資料: 6617492683
第 399 筆算術錯誤: -1032
第 400 筆錯誤資料: 6617495771
第 400 筆算術錯誤: 2056
第 401 筆錯誤資料: 6617491675
第 401 筆算術錯誤: -2040
第 402 筆錯誤資料: 6617495755
第 402 筆算術錯誤: 2040
第 403 筆錯誤資料: 6617491659
第 403 筆算術錯誤: -2056


第 693 筆錯誤資料: 6080622835
第 693 筆算術錯誤: -536870880
第 694 筆錯誤資料: 7154364595
第 694 筆算術錯誤: 536870880
第 695 筆錯誤資料: 6080622771
第 695 筆算術錯誤: -536870944
第 696 筆錯誤資料: 7691235571
第 696 筆算術錯誤: 1073741856
第 697 筆錯誤資料: 5543751923
第 697 筆算術錯誤: -1073741792
第 698 筆錯誤資料: 7691235507
第 698 筆算術錯誤: 1073741792
第 699 筆錯誤資料: 5543751859
第 699 筆算術錯誤: -1073741856
第 700 筆錯誤資料: 8764977395
第 700 筆算術錯誤: 2147483680
第 701 筆錯誤資料: 4470010099
第 701 筆算術錯誤: -2147483616
第 702 筆錯誤資料: 8764977331
第 702 筆算術錯誤: 2147483616
第 703 筆錯誤資料: 4470010035
第 703 筆算術錯誤: -2147483680
第 704 筆錯誤資料: 10912461043
第 704 筆算術錯誤: 4294967328
第 705 筆錯誤資料: 2322526451
第 705 筆算術錯誤: -4294967264
第 706 筆錯誤資料: 10912460979
第 706 筆算術錯誤: 4294967264
第 707 筆錯誤資料: 2322526387
第 707 筆算術錯誤: -4294967328
第 708 筆錯誤資料: 6617493907
第 708 筆算術錯誤: 192
第 709 筆錯誤資料: 6617493651
第 709 筆算術錯誤: -64
第 710 筆錯誤資料: 6617493779
第 710 筆算術錯誤: 64
第 711 筆錯誤資料: 6617493523
第 711 筆算術錯誤: -192
第 712 筆錯誤資料: 6617494035
第 712 筆算術錯誤: 320
第 713 筆錯誤資料: 6617493523
第 713 筆算術錯誤: -192
第 714 筆錯誤資料: 6617493907
第 

第 942 筆算術錯誤: 65280
第 943 筆錯誤資料: 6617427923
第 943 筆算術錯誤: -65792
第 944 筆錯誤資料: 6617625043
第 944 筆算術錯誤: 131328
第 945 筆錯誤資料: 6617362899
第 945 筆算術錯誤: -130816
第 946 筆錯誤資料: 6617624531
第 946 筆算術錯誤: 130816
第 947 筆錯誤資料: 6617362387
第 947 筆算術錯誤: -131328
第 948 筆錯誤資料: 6617756115
第 948 筆算術錯誤: 262400
第 949 筆錯誤資料: 6617231827
第 949 筆算術錯誤: -261888
第 950 筆錯誤資料: 6617755603
第 950 筆算術錯誤: 261888
第 951 筆錯誤資料: 6617231315
第 951 筆算術錯誤: -262400
第 952 筆錯誤資料: 6618018259
第 952 筆算術錯誤: 524544
第 953 筆錯誤資料: 6616969683
第 953 筆算術錯誤: -524032
第 954 筆錯誤資料: 6618017747
第 954 筆算術錯誤: 524032
第 955 筆錯誤資料: 6616969171
第 955 筆算術錯誤: -524544
第 956 筆錯誤資料: 6618542547
第 956 筆算術錯誤: 1048832
第 957 筆錯誤資料: 6616445395
第 957 筆算術錯誤: -1048320
第 958 筆錯誤資料: 6618542035
第 958 筆算術錯誤: 1048320
第 959 筆錯誤資料: 6616444883
第 959 筆算術錯誤: -1048832
第 960 筆錯誤資料: 6619591123
第 960 筆算術錯誤: 2097408
第 961 筆錯誤資料: 6615396819
第 961 筆算術錯誤: -2096896
第 962 筆錯誤資料: 6619590611
第 962 筆算術錯誤: 2096896
第 963 筆錯誤資料: 6615396307
第 963 筆算術錯誤: -2097408
第 964 筆錯誤資料: 6621688275
第 964 筆算術錯誤: 41

第 1192 筆算術錯誤: 10240
第 1193 筆錯誤資料: 6617487571
第 1193 筆算術錯誤: -6144
第 1194 筆錯誤資料: 6617499859
第 1194 筆算術錯誤: 6144
第 1195 筆錯誤資料: 6617483475
第 1195 筆算術錯誤: -10240
第 1196 筆錯誤資料: 6617512147
第 1196 筆算術錯誤: 18432
第 1197 筆錯誤資料: 6617479379
第 1197 筆算術錯誤: -14336
第 1198 筆錯誤資料: 6617508051
第 1198 筆算術錯誤: 14336
第 1199 筆錯誤資料: 6617475283
第 1199 筆算術錯誤: -18432
第 1200 筆錯誤資料: 6617528531
第 1200 筆算術錯誤: 34816
第 1201 筆錯誤資料: 6617462995
第 1201 筆算術錯誤: -30720
第 1202 筆錯誤資料: 6617524435
第 1202 筆算術錯誤: 30720
第 1203 筆錯誤資料: 6617458899
第 1203 筆算術錯誤: -34816
第 1204 筆錯誤資料: 6617561299
第 1204 筆算術錯誤: 67584
第 1205 筆錯誤資料: 6617430227
第 1205 筆算術錯誤: -63488
第 1206 筆錯誤資料: 6617557203
第 1206 筆算術錯誤: 63488
第 1207 筆錯誤資料: 6617426131
第 1207 筆算術錯誤: -67584
第 1208 筆錯誤資料: 6617626835
第 1208 筆算術錯誤: 133120
第 1209 筆錯誤資料: 6617364691
第 1209 筆算術錯誤: -129024
第 1210 筆錯誤資料: 6617622739
第 1210 筆算術錯誤: 129024
第 1211 筆錯誤資料: 6617360595
第 1211 筆算術錯誤: -133120
第 1212 筆錯誤資料: 6617757907
第 1212 筆算術錯誤: 264192
第 1213 筆錯誤資料: 6617233619
第 1213 筆算術錯誤: -260096
第 1214 筆錯誤資料: 661775

第 1442 筆錯誤資料: 6617739475
第 1442 筆算術錯誤: 245760
第 1443 筆錯誤資料: 6617215187
第 1443 筆算術錯誤: -278528
第 1444 筆錯誤資料: 6618034387
第 1444 筆算術錯誤: 540672
第 1445 筆錯誤資料: 6616985811
第 1445 筆算術錯誤: -507904
第 1446 筆錯誤資料: 6618001619
第 1446 筆算術錯誤: 507904
第 1447 筆錯誤資料: 6616953043
第 1447 筆算術錯誤: -540672
第 1448 筆錯誤資料: 6618558675
第 1448 筆算術錯誤: 1064960
第 1449 筆錯誤資料: 6616461523
第 1449 筆算術錯誤: -1032192
第 1450 筆錯誤資料: 6618525907
第 1450 筆算術錯誤: 1032192
第 1451 筆錯誤資料: 6616428755
第 1451 筆算術錯誤: -1064960
第 1452 筆錯誤資料: 6619607251
第 1452 筆算術錯誤: 2113536
第 1453 筆錯誤資料: 6615412947
第 1453 筆算術錯誤: -2080768
第 1454 筆錯誤資料: 6619574483
第 1454 筆算術錯誤: 2080768
第 1455 筆錯誤資料: 6615380179
第 1455 筆算術錯誤: -2113536
第 1456 筆錯誤資料: 6621704403
第 1456 筆算術錯誤: 4210688
第 1457 筆錯誤資料: 6613315795
第 1457 筆算術錯誤: -4177920
第 1458 筆錯誤資料: 6621671635
第 1458 筆算術錯誤: 4177920
第 1459 筆錯誤資料: 6613283027
第 1459 筆算術錯誤: -4210688
第 1460 筆錯誤資料: 6625898707
第 1460 筆算術錯誤: 8404992
第 1461 筆錯誤資料: 6609121491
第 1461 筆算術錯誤: -8372224
第 1462 筆錯誤資料: 6625865939
第 1462 筆算術錯誤: 8372224
第 1463 筆錯

第 1692 筆錯誤資料: 6618280147
第 1692 筆算術錯誤: 786432
第 1693 筆錯誤資料: 6617231571
第 1693 筆算術錯誤: -262144
第 1694 筆錯誤資料: 6617755859
第 1694 筆算術錯誤: 262144
第 1695 筆錯誤資料: 6616707283
第 1695 筆算術錯誤: -786432
第 1696 筆錯誤資料: 6618804435
第 1696 筆算術錯誤: 1310720
第 1697 筆錯誤資料: 6616707283
第 1697 筆算術錯誤: -786432
第 1698 筆錯誤資料: 6618280147
第 1698 筆算術錯誤: 786432
第 1699 筆錯誤資料: 6616182995
第 1699 筆算術錯誤: -1310720
第 1700 筆錯誤資料: 6619853011
第 1700 筆算術錯誤: 2359296
第 1701 筆錯誤資料: 6615658707
第 1701 筆算術錯誤: -1835008
第 1702 筆錯誤資料: 6619328723
第 1702 筆算術錯誤: 1835008
第 1703 筆錯誤資料: 6615134419
第 1703 筆算術錯誤: -2359296
第 1704 筆錯誤資料: 6621950163
第 1704 筆算術錯誤: 4456448
第 1705 筆錯誤資料: 6613561555
第 1705 筆算術錯誤: -3932160
第 1706 筆錯誤資料: 6621425875
第 1706 筆算術錯誤: 3932160
第 1707 筆錯誤資料: 6613037267
第 1707 筆算術錯誤: -4456448
第 1708 筆錯誤資料: 6626144467
第 1708 筆算術錯誤: 8650752
第 1709 筆錯誤資料: 6609367251
第 1709 筆算術錯誤: -8126464
第 1710 筆錯誤資料: 6625620179
第 1710 筆算術錯誤: 8126464
第 1711 筆錯誤資料: 6608842963
第 1711 筆算術錯誤: -8650752
第 1712 筆錯誤資料: 6634533075
第 1712 筆算術錯誤: 17039360
第 1713 筆

第 1941 筆算術錯誤: -58720256
第 1942 筆錯誤資料: 6676213971
第 1942 筆算術錯誤: 58720256
第 1943 筆錯誤資料: 6541996243
第 1943 筆算術錯誤: -75497472
第 1944 筆錯誤資料: 6760100051
第 1944 筆算術錯誤: 142606336
第 1945 筆錯誤資料: 6491664595
第 1945 筆算術錯誤: -125829120
第 1946 筆錯誤資料: 6743322835
第 1946 筆算術錯誤: 125829120
第 1947 筆錯誤資料: 6474887379
第 1947 筆算術錯誤: -142606336
第 1948 筆錯誤資料: 6894317779
第 1948 筆算術錯誤: 276824064
第 1949 筆錯誤資料: 6357446867
第 1949 筆算術錯誤: -260046848
第 1950 筆錯誤資料: 6877540563
第 1950 筆算術錯誤: 260046848
第 1951 筆錯誤資料: 6340669651
第 1951 筆算術錯誤: -276824064
第 1952 筆錯誤資料: 7162753235
第 1952 筆算術錯誤: 545259520
第 1953 筆錯誤資料: 6089011411
第 1953 筆算術錯誤: -528482304
第 1954 筆錯誤資料: 7145976019
第 1954 筆算術錯誤: 528482304
第 1955 筆錯誤資料: 6072234195
第 1955 筆算術錯誤: -545259520
第 1956 筆錯誤資料: 7699624147
第 1956 筆算術錯誤: 1082130432
第 1957 筆錯誤資料: 5552140499
第 1957 筆算術錯誤: -1065353216
第 1958 筆錯誤資料: 7682846931
第 1958 筆算術錯誤: 1065353216
第 1959 筆錯誤資料: 5535363283
第 1959 筆算術錯誤: -1082130432
第 1960 筆錯誤資料: 8773365971
第 1960 筆算術錯誤: 2155872256
第 1961 筆錯誤資料: 4478398675
第 1961 筆

In [13]:
DEC_R = []
for m in range(0,e2):
    mod = e2_data[m] % AN
    DEC_R.append(mod)

print("DEC_R的模數:\n",DEC_R)
print(np.size(DEC_R))

DEC_R的模數:
 [3, 6310, 1, 6308, 5, 6308, 3, 6306, 9, 6304, 7, 6302, 17, 6296, 15, 6294, 33, 6280, 31, 6278, 65, 6248, 63, 6246, 129, 6184, 127, 6182, 257, 6056, 255, 6054, 513, 5800, 511, 5798, 1025, 5288, 1023, 5286, 2049, 4264, 2047, 4262, 4097, 2216, 4095, 2214, 1882, 4431, 1880, 4429, 3763, 2550, 3761, 2548, 1214, 5099, 1212, 5097, 2427, 3886, 2425, 3884, 4853, 1460, 4851, 1458, 3394, 2919, 3392, 2917, 476, 5837, 474, 5835, 951, 5362, 949, 5360, 1901, 4412, 1899, 4410, 3801, 2512, 3799, 2510, 1290, 5023, 1288, 5021, 2579, 3734, 2577, 3732, 5157, 1156, 5155, 1154, 4002, 2311, 4000, 2309, 1692, 4621, 1690, 4619, 3383, 2930, 3381, 2928, 454, 5859, 452, 5857, 907, 5406, 905, 5404, 1813, 4500, 1811, 4498, 3625, 2688, 3623, 2686, 6, 6309, 2, 6305, 10, 6305, 6, 6301, 18, 6297, 14, 6293, 34, 6281, 30, 6277, 66, 6249, 62, 6245, 130, 6185, 126, 6181, 258, 6057, 254, 6053, 514, 5801, 510, 5797, 1026, 5289, 1022, 5285, 2050, 4265, 2046, 4261, 4098, 2217, 4094, 2213, 1883, 4432, 1879, 4428, 3764,

In [14]:
total_AWE = AWE1_1 + AWE_2
e_total = e1_data1 + e2_data
R_total = SEC_R + DEC_R
print("還未去除重複error data的數量:",np.size(e_total))
print("還未去除重複餘數的數量:", np.size(R_total))

## 把重複的 AWE 及 error_data 去除
AWE = remove_duplicates(total_AWE)
new_e_total = remove_duplicates(e_total)
new_R_total = remove_duplicates(R_total)
error_total_count = np.size(new_e_total)
print("已去除重複error data的數量:",error_total_count)
print("已未去除重複餘數的數量:",np.size(new_R_total))
print("已未去除重複AWE的數量:",np.size(AWE))
print("所有的AWE為:\n",AWE)
print("所有算術錯誤的資料:\n", new_e_total)

還未去除重複error data的數量: 2178
還未去除重複餘數的數量: 2178
已去除重複error data的數量: 2052
已未去除重複餘數的數量: 2052
已未去除重複AWE的數量: 2052
所有的AWE為:
 [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 3, -3, 5, -5, 9, -7, 7, -9, 17, -15, 15, -17, 33, -31, 31, -33, 65, -63, 63, -65, 129, -127, 127, -129, 257, -255, 255, -257, 513, -511, 511, -513, 1025, -1023, 1023, -1025, 2049, -2047, 2047, -2049, 4097, -4095, 4095, -4097, 8193, -8191, 8191, -8193, 16385, -16383, 16383, -16385, 32769, -32767, 32767, -32769, 65537, -65535, 65535, -65537, 131073, -131071, 13107

### 所有AWE數量: $4*C^{bits}_{2}$ $-$ $2*(bits-1)$ - $2*(bits-2)$
* $2*(bits-1)$: 所有的 single AWE都可以分解成已重複的 double AWE, 除了MSB例外
* EX: $(+,0)$ 只能分解成 $(0,++)$ , $(-,0)$ 只能分解成 $(0,--)$
* $2*(bits-2)$: double AWE重複的個數
* EX: $(+,0,-)$ = $(0,+,+)$ and $(-,0,+)$ = $(0,-,-)$

### Trade-off Algorithm: 2025/05/27 使用 Vivado HLX IP 驗證
* 此解出來的數字為位置及正負號 EX: $-1$ 代表 $-2^0$ , $-5$ 代表 $-2^4$ 以此類推

In [16]:
import math
import time
### 給定一個DEC_R, 找錯誤位置h1及h2(包含錯誤正負號)
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]

start_time = time.time()  # 記錄開始時間
for R, error in zip(new_R_total, new_e_total): 
    ip.write_mm(0, R)
    
    h1 = twos_complement_32bit(ip.read_mm(4))                  
    h2 = twos_complement_32bit(ip.read_mm(8))
    
    s2 = get_sign(h2)
    s1 = get_sign(h1)
    
    possible_h1.append(h1)                  
    possible_h2.append(h2)                  
    output.append((error - (s1 * 2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // AN)

            
end_time = time.time()  # 記錄結束時間
elapsed_time = end_time - start_time  # 計算執行時間
print(f"執行時間: {elapsed_time:.6f} 秒")
print("更正後的值是否和原data相同:", data1 == output.pop())
print(elapsed_time / np.size(possible_h1))
## print(output)
### print(possible_h1)
### print(possible_h2)
##  print("更正後的值:",output)
print(np.size(possible_h1))
print(np.size(possible_h2))
##print("輸入的 error data的總數是否和經過演算法後相同:", error_total_count == np.size(possible_h1))
##print(possible_SE_h1)

執行時間: 0.220855 秒
更正後的值是否和原data相同: True
0.00010762902495921472
2052
2052
